<h1><span style="color:red">Recoding variables using DDI 2.0/2.5 metadata</span></h1>

Using this notebook, you can import a DDI codebook and use it to reformat categorical variables, with the option to recode question labels. Then the notebook will let you create a new SuAVE survey with the updated survey file. Currently this notebook does not check for vargrp tags in the DDI codebook.

Author: John Chen
(Adapted from ordinal recode by Iakov Vasilyev)


## 1. Retrieve survey parameters from the URL

In [ ]:
%%javascript
function getQueryStringValue (key)
{  
    return unescape(window.location.search.replace(new RegExp("^(?:.*[&\\?]" + escape(key).replace(/[\.\+\*]/g, "\\$&") + "(?:\\=([^&]*))?)?.*$", "i"), "$1"));
}
IPython.notebook.kernel.execute("survey_url='".concat(getQueryStringValue("surveyurl")).concat("'"));
IPython.notebook.kernel.execute("views='".concat(getQueryStringValue("views")).concat("'"));
IPython.notebook.kernel.execute("view='".concat(getQueryStringValue("view")).concat("'"));
IPython.notebook.kernel.execute("user='".concat(getQueryStringValue("user")).concat("'"));
IPython.notebook.kernel.execute("csv_file='".concat(getQueryStringValue("csv")).concat("'")); 
IPython.notebook.kernel.execute("dzc_file='".concat(getQueryStringValue("dzc")).concat("'")); 
IPython.notebook.kernel.execute("params='".concat(getQueryStringValue("params")).concat("'")); 
IPython.notebook.kernel.execute("active_object='".concat(getQueryStringValue("activeobject")).concat("'")); 
IPython.notebook.kernel.execute("full_notebook_url='" + window.location + "'"); 

## 2. Import libraries

In [ ]:
#common imports
from __future__ import print_function
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import Markdown, display

import param
import panel as pn
import pandas as pd
import numpy as np
import xml.etree.ElementTree as ET

pn.extension()

def printmd(string):
    display(Markdown(string))

absolutePath = "../../temp_csvs/"
codebookPath = "../../temp_csvs"#temp_codebooks/"

# local imports
import sys
sys.path.insert(1, '../../helpers')
import panel_libs as panellibs
import suave_integration as suaveint

url_partitioned = full_notebook_url.partition('/operations')
base_url = url_partitioned[0];

import re

In [ ]:
def slider(data):
    """
    slider creates an interactive display of a
    data frame.
    
    :param df: data frame
    :returns: interactive dataframe
    """
    
    ## Row Selector widget
    row_selection = pn.widgets.IntSlider(name='Navigate Rows', width=350, 
                                         margin=(0,50,-15,0), end=len(df)-1)

    # Column Selector widget
    col_selection = pn.widgets.IntSlider(name='Navigate Columns', width=350, 
                                         margin=(0,0,5,0), end=len(df.columns))
    
    @pn.depends(row_selection.param.value, col_selection.param.value)
    def navigate_data(row=0, col=0):
        return data.iloc[row:row+5, col:col+10]
    
    sliders = pn.Row(row_selection, col_selection, margin=(0,0,0,10))
    full_widget = pn.Column(sliders, navigate_data)
    return full_widget

def extract_url(url, file_type):
    """
    extract_url reads a file provided as a url link
    
    :param url: string representing url link to file
    :param file_type: string representing file extension, .csv or .tsv
    :returns: data frame of file
    """
    if file_type == 'csv':
        try:
            data = pd.read_csv(url, sep=',', encoding="latin-1")
        except UniodeDecodeError:
            data = pd.read_csv(url, sep=',', encoding="ISO-8859-1")
    elif file_type == 'tsv':
        try:
            data = pd.read_csv(url, sep='\t', encoding="latin-1")
        except UniodeDecodeError:
            data = pd.read_csv(url, sep='\t', encoding="ISO-8859-1")
    else:
        return None
    
    return data

def extract_data(path):
    """
    extract_data reads files from various formats
    
    :param link: string representing path to file
    :returns: data frame of file
    """

    # Reading file at path
    if path.endswith(('.txt', 'tsv')):
        try:
            data = pd.read_csv(path, sep='\t', encoding="latin-1")
        except UnicodeDecodeError:
            data = pd.read_csv(path, sep='\t', encoding="ISO-8859-1")
    elif path.endswith('.csv'):
        try:
            data = pd.read_csv(path, encoding="latin-1")
        except UnicodeDecodeError:
            data = pd.read_csv(path, encoding="ISO-8859-1")
    elif path.endswith('.sav'):
        try:
            data = pd.read_spss(path, encoding="latin-1")
        except UnicodeDecodeError:
            data = pd.read_csv(path, encoding="ISO-8859-1")
    else:
        return None
    
    return data

def extract_xml(path):
    """
    extract_xml reads in an xml formatted file
    :param path: string representing path to file
    :returns: element tree of xml hierarchy
    """
    if path.endswith('.xml'):
        tree = ET.parse(path)
    else:
        return None
    
    return tree

## 3. Import DDI codebook

In [ ]:
#widget for file upload
ddi_input = pn.widgets.FileInput(accept='.xml')
ddi_select = pn.widgets.RadioBoxGroup(name='Select DDI version', options=['DDI 2.0/DDI Lite', 
                                                                        'DDI 2.5'], 
                                       inline=False)
    
pn.Column('Upload metadata and proceed to step 4' , ddi_input, ddi_select)



In [ ]:
ns = {}
global xmlname

if ddi_select.value == 'DDI 2.0/DDI Lite':
    ns = {'ddi': 'http://www.icpsr.umich.edu/DDI'}
else:
    ns = {'ddi': 'ddi:codebook:2_5'}
    
if not pd.isnull(ddi_input.filename):
    xmlname = codebookPath + ddi_input.filename
    ddi_input.save(xmlname)

varData = extract_xml(xmlname)
dataDscr = varData.find('ddi:dataDscr', ns)


## 4. Select a survey file from SuAVE, import a local CSV file, or get survey from url

In [ ]:
data_select = pn.widgets.RadioBoxGroup(name='Select notebook', options=['Load survey file from SuAVE', 
                                                                        'Import a local CSV file', 'Get survey from url'], 
                                       inline=False)
data_select

In [ ]:
data_input = pn.widgets.FileInput()
url_input = pn.widgets.TextInput()
url_survey_format = pn.widgets.RadioBoxGroup(name='Get filetype', options=['csv', 
                                                                        'tsv'], 
                                       inline=False)
    
def check_selection():
    if data_select.value == 'Load survey file from SuAVE':
        global fname
        fname = absolutePath + csv_file
        printmd("<b><span style='color:red'>SuAVE survey will be loaded. Continue to step 5.</span></b>")
        return

    elif data_select.value == 'Import a local CSV file':
        message = pn.pane.HTML("<b><span style='color:red'>Upload data and continue to step 5.</span></b>")
        return pn.Column(message, data_input)
    else:
        message = pn.pane.HTML("<b><span style='color:red'>Input survey url here, press Enter, and then run the next cell:</span></b>")
        return pn.Column(message, url_input, url_survey_format)
    
check_selection()

In [ ]:
df = pd.DataFrame()
if not pd.isnull(data_input.filename):
    fname = absolutePath + data_input.filename
    data_input.save(fname)
    df = extract_data(fname)
elif not pd.isnull(url_input.value):
    df = extract_url(url_input.value, url_survey_format.value)

#for testing only
#df = df.head(100) 

## 5. Recode categorical data values using DDI codebook

In [ ]:
#Loop through every variable defined in codebook
for var in dataDscr.findall('ddi:var', ns):
    name = var.attrib['name']
    varformat = var.find("ddi:varFormat", ns).attrib['type'] 
    catgrytrue = ET.iselement(var.find("ddi:catgry", ns))
    if catgrytrue and varformat == 'numeric':
        
        #If variable is categorical, create a dictionary matching numeric value to label
        catdict = {}
        for cat in var.findall('ddi:catgry', ns):
            catVal = cat[0].text.strip(' \n\t')
            catdict[int(catVal)] = catVal + " " + cat[1].text.strip(' \n\t')
            
        #Replace values using dictionary
        df = df.replace({name : catdict})

#display survey values
updated_df = df.copy()
panellibs.slider(updated_df)


## 6. Edit variable names

Note: If question label was improperly defined in the codebook, they may be poorly formatted or too long to display in SuAVE or poorly formatted. In that case, some manual cleanup may be required

In [ ]:
#create dataframe with questions and their labels

#get question labl text from codebook
qstnlabl = []
for var in dataDscr.findall('ddi:var', ns):
    labl = var.find('ddi:labl', ns)
    if labl is not None:
        qstnlabl.append(labl.text)
    else:
        qstnlabl.append(None)

        
#convert to dataframe using var name as index and question label as value
qstndf = pd.DataFrame({'questions':qstnlabl}, index=[var.attrib['name'] for var in dataDscr.findall('ddi:var', ns)])

#display question values
panellibs.slider(qstndf)


## 6a. Create question groupings

If vargrp wasn't defined in the codebook, this widget allows for manual selection of groups, which may help in editing question values

In [ ]:
#Question group panel
# Left panel
left_text = pn.Row("####Select Variables", margin=(0,0,-15,270))
group_selector = pn.widgets.CrossSelector(options=list(df.columns), width=630)
left_panel = pn.Column(left_text, group_selector, css_classes=['widget-box'], margin=(0,30,0,0))

# Right panel
group_text = pn.Row("####Specify Group Name", margin=(0,0,-20, 25))
group_name = pn.widgets.TextInput(placeholder='Enter Name', width=200, margin=(11,10,10,10))
name_input = pn.Column(group_text, group_name, margin=(-5,0,0,0))
right_panel = pn.Column(name_input, css_classes=['widget-box'], width=225, height=150)

apply_button = pn.widgets.Toggle(name='Create Mapping', button_type='primary',  margin=(30,0,10,10), width=200)

# Groupings display
group_text = pn.pane.Markdown('#### Groupings ', width=950)

Groupings = {}
#condition_values = []
@pn.depends(group_selector.param.value, apply_button.param.value)
def group(b_var, apply):
    apply_button.value = False
    
    if (b_var != []):
        
        # Determines possible conditions based on variables selected
        possible_conditions = pd.unique(df[b_var].values.ravel('K'))
        if (group_name.value != '') and apply:
            
            # Creates mapping
            new_group = group_name.value
            Groupings[new_group] = b_var
            #condition_values.append(condition.value)
            
            # Refreshes selected group values and name
            group_selector.value = []
            group_name.value = ''
            
            # Updates grouping display
            new_mapping = '- **'+ str(new_group) + '**' +' &rarr; '+ str(Groupings[new_group])
            group_text.object = group_text.object + '\n' + new_mapping
            
            return group_text
        
    return group_text

# Display widgets
widgets = pn.Row(left_panel, pn.Column(right_panel, apply_button))
full_display = pn.Column(widgets, group)
full_display

<b><span style="color:red">Workspace for additional formatting changes</span></b>

In [ ]:
'''
Key variables:
  Groupings provides names of variables within each defined group
  qstndf is a Dataframe of questions labeled by var name
'''

#Example:
#get all questions from group 'Q3', and save as a dataframe
Q3_df = qstndf.loc[[var for var in Groupings['Q3']]]

#remove first 20 characters from each question in Q3_df
#Q3_df

In [ ]:
#replace var name with questions
updated_df.columns = qstndf['questions'].tolist()
panellibs.slider(updated_df)

## 7. Generate a new survey and open it in SuAVE

In [ ]:
# new filename

if data_select.value == 'Import a local CSV file':
    csv_file = data_input.filename
df = updated_df
new_file = absolutePath + csv_file[:-4]+'_v1.csv'
printmd("<b><span style='color:red'>A new temporary file will be created at: </span></b>")
print(new_file)
df.to_csv(new_file, index=None)

In [ ]:
#Input survey name

input_text = widgets.Text()
output_text = widgets.Text()

def bind_input_to_output(sender):
    output_text.value = input_text.value

# Tell the text input widget to call bind_input_to_output() on submit
input_text.on_submit(bind_input_to_output)

printmd("<b><span style='color:red'>Input survey name here, press Enter, and then run the next cell:</span></b>")
# Display input text box widget for input
display(input_text)

display(output_text)

In [ ]:
#Print survey name
survey_name = output_text.value
printmd("<b><span style='color:red'>Survey Name is: </span></b>" + survey_name)

In [ ]:
referer = survey_url.split("/main")[0] +"/"
upload_url = referer + "uploadCSV"
new_survey_url_base = survey_url.split(user)[0]

import requests
import re
csv = {"file": open(new_file, "rb")}

if data_select.value == 'Import a local CSV file':
    dzc_file = ''
    views = '1110001'
    view='grid'

upload_data = {
    'name': input_text.value,
    'dzc': dzc_file,
    'user':user
}
headers = {
    'User-Agent': 'suave user agent',
    'referer': referer
}

r = requests.post(upload_url, files=csv, data=upload_data, headers=headers)

if r.status_code == 200:
    printmd("<b><span style='color:red'>New survey created successfully</span></b>")
    regex = re.compile('[^0-9a-zA-Z_]')
    s_url = survey_name
    s_url =  regex.sub('_', s_url)

    url = new_survey_url_base + user + "_" + s_url + ".csv" + "&views=" + views + "&view=" + view
    print(url)
    printmd("<b><span style='color:red'>Click the URL to open the new survey</span></b>")
else:
    printmd("<b><span style='color:red'>Error creating new survey. Check if a survey with this name already exists.</span></b>")
    printmd("<b><span style='color:red'>Reason: </span></b>"+ str(r.status_code) + " " + r.reason)